In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import time
from datetime import datetime
from requests.exceptions import RequestException
import xml.etree.ElementTree as ET

In [8]:
import pandas as pd
import re
import xml.etree.ElementTree as ET
from datetime import datetime

class DataProcessor:
    """Class for processing XML data and cleaning nutritional data."""

    def __init__(self):
        """Initialize the DataProcessor class."""
        pass  # No instance variables needed, as all methods are static

    @staticmethod
    def clean_nutrition_data(df):
        """
        Cleans the nutrition data by:
        - Removing 'g', 'mg', 'micro', 'perc', and '°C'
        - Converting appropriate units to float

        Args:
            df (pd.DataFrame): DataFrame containing nutritional data.

        Returns:
            pd.DataFrame: Cleaned DataFrame.
        """
        excluded_indexes = [0, 1, 3, 4, 5, 10]
        excluded_columns = ['Fogás', 'Konyha', 'Nehézség', 'Elkészítési idő', 'Szakács elkészítette', 
                            'Speciális étrendek', 'Vegetáriánus', 'Alkalom', 'Költség egy főre', 
                            'Konyhatechnológia']
        
        for index, col in enumerate(df.columns):
            if index in excluded_indexes or col in excluded_columns:
                continue
            if df[col].dtype == 'object':
                df[col] = df[col].apply(lambda x: DataProcessor.clean_value(x))
        
        return df
   
    @staticmethod
    def clean_nutrition_data_ingredients(df):
        """
        Cleans numerical nutrition data by:
        - Removing units like 'g', 'mg', 'µg', 'micro'
        - Converting to float and scaling appropriately
        - Keeping non-numeric columns unchanged

        Args:
        df (pd.DataFrame): The dataframe containing nutritional data.

        Returns:
        pd.DataFrame: Cleaned dataframe with corrected values.
        """
        excluded_columns = ['Alapanyag neve', 'Elsődleges kategória', 'Másodlagos kategória']

        for col in df.columns:
            if col in excluded_columns:
                continue  # Skip non-numeric columns
            if df[col].dtype == 'object':
                df[col] = df[col].apply(lambda x: clean_value(x))
        return df
    
    @staticmethod
    def clean_value(value):
        """
        Cleans individual value by:
        - Removing unwanted characters
        - Converting to appropriate numeric format

        Args:
            value (str): The value to clean.

        Returns:
            float or str: Cleaned numeric value if applicable, otherwise original string.
        """
        if isinstance(value, str):
            value = value.strip().lower()
            has_number = any(char.isdigit() for char in value)

            if has_number:
                if 'mg' in value:
                    return float(value.replace('mg', '').strip()) / 1000
                elif 'µg' in value:
                    return float(value.replace('µg', '').strip()) / 1_000_000
                elif 'micro' in value:
                    return float(value.replace('micro', '').strip()) / 1_000_000
                elif 'g' in value:
                    return float(value.replace('g', '').strip())
                elif 'perc' in value:
                    return float(value.replace('perc', '').strip())
                elif '°c' in value:
                    return float(value.replace('°c', '').strip())

        return value  # Return unchanged if not numeric

    @staticmethod
    def clean_ingredient_value(value):
        """
        Cleans an individual value by:
        - Removing units ('g', 'mg', 'µg', 'micro', 'mcg')
        - Converting to float and scaling appropriately
        - Handling cases where unexpected characters appear

        Args:
        value (str): The original value.

        Returns:
        float or original value if conversion fails.
        """
        if isinstance(value, str):
            value = value.strip().lower()

            # Extract numeric part using regex
            num_match = re.search(r"[-+]?\d*\.?\d+", value)
            if not num_match:
                return value  # Return as-is if no number is found

            num = float(num_match.group())  # Extract the numerical value

            # Check for unit and scale accordingly
            if "mg" in value:
                return num / 1000  # Convert mg to g
            elif "µg" in value or "mcg" in value or "micro" in value:
                return num / 1000000  # Convert µg/micro/mcg to g
            elif "g" in value:
                return num  # Already in g, no change needed

            return num  # If no unit is found, assume the number is correct

        return value  # Return original if not a string


In [21]:
class RecipeScraper:
    """Class for scraping full details of a recipe from Nosalty."""

    BASE_URL = "https://www.nosalty.hu/recept/"

    def __init__(self, url):
        self.url = url
        self.soup = self.get_soup()
        
    def extract_urls_from_xml(file_path, check_url_format=False, date_range=None):
        """
        Extracts recipe URLs from an XML file, optionally checking format and date range.
        
        Args:
            file_path (str): Path to the XML file.
            check_url_format (bool): If True, checks if the URL is valid.
            date_range (tuple): Optional (start_date, end_date) for filtering URLs by lastmod.
                                Format: ('YYYY-MM-DD', 'YYYY-MM-DD').

        Returns:
            list: List of valid URLs.
        """
        def is_valid_url(url):
            """Validates URL format."""
            pattern = r"^(http|https)://[^\s/$.?#].[^\s]*$"
            return re.match(pattern, url) is not None

        def is_within_date_range(date_text, start_date, end_date):
            """Checks if the date falls within the specified range."""
            try:
                date = datetime.strptime(date_text, "%Y-%m-%d")
                return start_date <= date <= end_date
            except ValueError:
                return False

        # Define XML namespace
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

        # Parse XML
        tree = ET.parse(file_path)
        root = tree.getroot()

        urls = []
        base_url = "https://www.nosalty.hu/recept/"
        for url in root.findall('ns:url', namespace):
            loc = url.find('ns:loc', namespace)
            lastmod = url.find('ns:lastmod', namespace)

            if loc is not None:
                url_text = loc.text
                if not url_text.startswith(base_url):
                    continue
                if check_url_format and not is_valid_url(url_text):
                    continue
                if date_range and lastmod is not None:
                    start_date, end_date = map(lambda d: datetime.strptime(d, "%Y-%m-%d"), date_range)
                    if not is_within_date_range(lastmod.text, start_date, end_date):
                        continue
                urls.append(url_text)

        return urls
    
    def get_soup(self):
        """Fetch and parse the webpage."""
        try:
            response = requests.get(self.url, timeout=10)
            if response.status_code != 200:
                return None
            return BeautifulSoup(response.text, 'html.parser')
        except requests.RequestException:
            return None

    def get_title(self):
        """Extracts the recipe title, or returns None if unavailable."""
        title_tag = self.soup.find('h1', class_='p-article__title -recipe pt-10 px-3 mb-5 d-block')
        return title_tag.text.strip() if title_tag else 'N/A' 

    def get_additional_info(self):
        """Extracts additional metadata related to the recipe."""
        info = []
        classes = [
            'a-text -fontSize-9 -fontColorSenary d-none d-sm-block',
            'a-text -fontSize-9 -fontColorQuaternary d-none d-sm-block',
            'a-link -fontColorPrimary -fontSize-16 -recipe pr-3 pl-2'  # Now capturing <a> elements
        ]

        for class_name in classes:
            elements = self.soup.find_all(['span', 'a'], class_=class_name)  # Look for both <span> and <a>
            for element in elements:
                info.append(element.text.strip())

        return ', '.join(info)

    def get_recipe_details(self):
        """Extracts the available recipe details: Time, Cost, and Difficulty."""
        details = {}

        # Define mapping from Hungarian labels to English keys
        rename_map = {
            "Idő": "Time",
            "Költség": "Cost",
            "Nehézség": "Difficulty"
        }

        # Locate all sections containing the relevant details
        detail_sections = self.soup.find_all('div', class_='p-recipe__detailsBody d-print-flex justify-content-end text-center mb-1 w-100')

        for section in detail_sections:
            label_tag = section.find('span', class_='p-recipe__detailsHeading d-block text-uppercase')  # Find label
            value_tag = section.find('div') or section.find('time') or section.find('span')  # Find actual value

            if label_tag and value_tag:
                label_text = label_tag.text.strip()
                value_text = value_tag.text.strip()

                # Store the extracted data with correct English labels
                if label_text in rename_map:
                    details[rename_map[label_text]] = value_text  # Correctly stores the value now

        return details

    
    def get_preparation_and_baking_time(self):
        """Extracts preparation and baking time from the recipe details list."""
        details = {}

        # Define label mapping from Hungarian to English
        rename_map = {
            "Előkészítés ideje": "Preparation Time",
            "Sütés ideje": "Baking Time"
        }

        # Locate the container that holds preparation and baking times
        details_section = self.soup.find('div', class_='p-recipe__detailsList')
        if not details_section:
            return details  # Return empty if section is missing

        # Find the list containing preparation and baking times
        list_section = details_section.find('ul', class_='m-list__list')
        if not list_section:
            return details  # Return empty if no list is found

        # Find all list items with the relevant class
        list_items = list_section.find_all('li', class_='m-list__item -simple -fontSize-18')

        for item in list_items:
            # Find the label (first span)
            label_tag = item.find('span')
            # Find the value (last span in the item)
            value_tag = item.find_all('span')[-1] if item.find_all('span') else None

            if label_tag and value_tag:
                label_text = label_tag.text.strip()
                value_text = value_tag.text.strip()

                # Store data using mapped labels
                if label_text in rename_map:
                    details[rename_map[label_text]] = value_text

        return details

    def get_portions(self):
        """Extracts the portion size from the input field."""

        # Find portions (now inside an input field with id="adag")
        portion_input = self.soup.find('input', {'id': 'adag'})
        portion_value = portion_input['value'].strip() if portion_input and 'value' in portion_input.attrs else 'N/A'

        return portion_value

    def get_nutrition_data(self):
        """Extracts all available nutrition details (Protein, Fat, Carbs, Water, and Minerals)."""
        nutrition_data = {}

        # Define the section titles and their corresponding nutrient names
        nutrition_sections = {
            'Fehérje': ['Összesen'],  # Protein
            'Zsír': ['Összesen', 'Telített zsírsav', 'Egyszeresen telítetlen zsírsav', 'Többszörösen telítetlen zsírsav', 'Koleszterin'],  # Fats
            'Szénhidrátok': ['Összesen', 'Cukor', 'Élelmi rost'],  # Carbs
            'VÍZ': ['Összesen'],  # Water
            'Ásványi anyagok': ['Cink', 'Szelén', 'Kálcium', 'Vas', 'Magnézium', 'Foszfor', 'Nátrium', 'Réz', 'Mangán'],  # Minerals
            'Vitaminok': ['A vitamin', 'B6 vitamin', 'B12 Vitamin', 'E vitamin', 'C vitamin', 'D vitamin', 'K vitamin',
                          'Tiamin - B1 vitamin', 'Riboflavin - B2 vitamin', 'Niacin - B3 vitamin', 'Pantoténsav - B5 vitamin',
                          'Folsav - B9-vitamin', 'Kolin', 'Retinol', 'α-karotin', 'β-karotin', 'β-crypt', 'Likopin', 'Lut-zea']  # Vitamins
        }

        # Loop through each section and extract relevant data
        for section, nutrients in nutrition_sections.items():
            tag = self.soup.find('h4', string=section)  # Find the section header (e.g., "Fehérje")

            if tag:
                list_section = tag.find_next('ul')  # Get the next <ul> after the section header

                if list_section:
                    for li in list_section.find_all('li'):  # Iterate through each <li> inside the section
                        for nutrient in (nutrients if isinstance(nutrients, list) else [nutrients]):
                            if nutrient in li.text:  # If the nutrient name appears in the text
                                value = li.find('div', class_='kc')  # Find the actual value
                                value_text = value.text.strip() if value else 'N/A'

                                # **Fix "Összesen" overwriting issue** by renaming it based on its section
                                if nutrient == "Összesen":
                                    renamed_nutrient = f"{section} - Összesen"  # Example: "Fehérje - Összesen"
                                else:
                                    renamed_nutrient = nutrient  # Keep other names unchanged

                                # Store the correctly named value
                                nutrition_data[renamed_nutrient] = value_text

        return nutrition_data


    def get_ingredients(self):
        """Extracts ingredient names, their full URLs, and amounts."""
        ingredients_list = []

        # Locate the correct section in the HTML
        ingredients_section = self.soup.find('ul', class_='m-list__list -nutrition')

        if ingredients_section:
            items = ingredients_section.find_all('li', class_='m-list__item p-2 -dotted -fontSize-18 d-flex justify-content-between pl-5')

            for item in items:
                # Extract quantity from span
                quantity_tag = item.find('span', class_='-fontSize-lg-16')
                quantity = quantity_tag.text.strip() if quantity_tag else 'N/A'

                # Extract ingredient name and URL from <a> tag
                ingredient_tag = item.find('a', class_='a-link -primaryHoverEffect -fontSize-18 -fontSize-lg-16')
                if ingredient_tag:
                    ingredient_name = ingredient_tag.text.strip()
                else:
                    ingredient_name = 'N/A'

                # Append as dictionary
                ingredients_list.append({
                    "Ingredient": ingredient_name,
                    "Quantity": quantity
                })
    
        return ingredients_list


    def to_dataframe(self):
        """Converts extracted recipe details to a pandas DataFrame."""
        return pd.DataFrame([{
            'Recipe name': self.get_title(),
             **self.get_recipe_details(),
            'Categories': self.get_additional_info(),
            **self.get_preparation_and_baking_time(),
            'Portions': self.get_portions(),           
            'Ingredients': self.get_ingredients(),
            **self.get_nutrition_data()
        }])

In [22]:
# Defining XML file paths
file_paths = [
    r'C:\Users\Bálint\Desktop\Asztal\Projekt\inputs\receptek_url.xml',
    r'C:\Users\Bálint\Desktop\Asztal\Projekt\inputs\receptek_url_2.xml',
    r'C:\Users\Bálint\Desktop\Asztal\Projekt\inputs\receptek_url_3.xml'
]

# Processing all URLs
all_urls = []
for file_path in file_paths:
    urls = RecipeScraper.extract_urls_from_xml(file_path, check_url_format=True)
    all_urls.extend(urls)

In [24]:
import traceback
# Start time tracking
start_time = time.time()

all_recipe_details = []
# Initialize counters
success_count = 0
total_count = len(all_urls)

# Process each recipe URL
with tqdm(total=total_count, desc="Processing Recipes") as pbar:
    for i, url in enumerate(all_urls, start=1):
        try:
            scraper = RecipeScraper(url)
            recipe_details_df = scraper.to_dataframe()

            # **Check if recipe details are valid**
            if recipe_details_df is not None:
                all_recipe_details.append(recipe_details_df)
                success_count += 1  # Count only successful scrapes

        except Exception as e:
            print(f"Error while processing {url} ")
        
        # Periodically log progress
        if i % 5000 == 0 or i == total_count:
            elapsed_time = time.time() - start_time
            print(f"Processed {i}/{total_count} URLs. Successful recipes: {success_count}. Elapsed time: {elapsed_time:.2f} seconds.")
        
        pbar.update(1)

# Combine all extracted data into a single DataFrame
final_df = pd.concat(all_recipe_details, ignore_index=True)
final_df_cleaned = DataProcessor.clean_nutrition_data(final_df)

# Save the cleaned results to a CSV file
output_path = r"C:\Users\Bálint\Desktop\Asztal\Projekt\receptek_jav.csv"
final_df_cleaned.to_csv(output_path, index=False)

# End time tracking
end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")
# Display the first few rows of the cleaned dataset
final_df_cleaned.head()

Processing Recipes:   5%|██▋                                                 | 3614/71260 [1:08:11<16:59:08,  1.11it/s]

Error while processing https://www.nosalty.hu/recept/olga-almas 


Processing Recipes:   5%|██▋                                                 | 3648/71260 [1:08:50<14:24:23,  1.30it/s]

Error while processing https://www.nosalty.hu/recept/hideg-gorog-gyumolcsleves-fozes-nelkul 


Processing Recipes:   5%|██▋                                                 | 3749/71260 [1:10:45<15:37:59,  1.20it/s]

...

Processing Recipes: 100%|███████████████████████████████████████████████████▉| 71185/71260 [23:41:46<00:00, 124.83it/s]

Error while processing https://www.nosalty.hu/recept/konnyu-kraterszelet 
Error while processing https://www.nosalty.hu/recept/kuglof-tom-cruise-kokusz-fehercsoki 
Error while processing https://www.nosalty.hu/recept/fahej-keksz-ciniminis 
Error while processing https://www.nosalty.hu/recept/aszalt-szilvaval-gongyolt-csirkemell-baconkontos 
Error while processing https://www.nosalty.hu/recept/pulyka-wellington-modra 
Error while processing https://www.nosalty.hu/recept/dios-szilvas-papucs-karacsony-sutemeny-aprosuti-auchan-vendegvaro 
Error while processing https://www.nosalty.hu/recept/mikulas-suti 
Error while processing https://www.nosalty.hu/recept/dios-makos-bejgliszelet 
Error while processing https://www.nosalty.hu/recept/legegyszerubb-puspokkenyer 
Error while processing https://www.nosalty.hu/recept/grillazs-hazilag 
Error while processing https://www.nosalty.hu/recept/egyszeru-lazac 
Error while processing https://www.nosalty.hu/recept/lazacos-rakott-teszta 
Error while proce

Processing Recipes: 100%|█████████████████████████████████████████████████████| 71260/71260 [23:41:46<00:00,  1.20s/it]


Error while processing https://www.nosalty.hu/recept/sajtos-karfiolnuggets 
Error while processing https://www.nosalty.hu/recept/narancslekvar-marmalade-narancsdzsem 
Error while processing https://www.nosalty.hu/recept/kevert-salata-savanyu-kaposztaval 
Error while processing https://www.nosalty.hu/recept/kokuszos-puncsgolyo-piritott-mandulaval 
Error while processing https://www.nosalty.hu/recept/turoval-toltott-csoroge-cannoli 
Error while processing https://www.nosalty.hu/recept/egyszeru-csorogefank 
Error while processing https://www.nosalty.hu/recept/vegan-csokitorta-egyszeruen 
Error while processing https://www.nosalty.hu/recept/brownie-sajttorta-eper-valentinnap-sutemeny 
Error while processing https://www.nosalty.hu/recept/churros-csokiszosszal 
Error while processing https://www.nosalty.hu/recept/paradicsomos-babfozelek-villamgyorsan 
Error while processing https://www.nosalty.hu/recept/citrompite-fank 
Error while processing https://www.nosalty.hu/recept/klasszikus-skot-toj

,Recipe name,Time,Cost,Difficulty,Categories,Preparation Time,Baking Time,Portions,Ingredients,Fehérje - Összesen,...,Niacin - B3 vitamin,Pantoténsav - B5 vitamin,Folsav - B9-vitamin,Kolin,Retinol,α-karotin,β-karotin,β-crypt,Likopin,Lut-zea
0,Tőzegáfonyás répatorta,50perc,olcsó,könnyű,"torta, répatorta",5 perc,45.0,8,"[{'Ingredient': 'nádcukor', 'Quantity': '150 g...",4.8,...,0.000,0.0,0.000018,0.058,0.000029,0.000619,0.001478,0.000002,0.0,0.000149
1,Hideg joghurtos őszibarackleves,51perc,olcsó,könnyű,"levesek, gyümölcsleves",6 perc,45.0,4,"[{'Ingredient': 'őszibarack', 'Quantity': '8 k...",4.2,...,0.002,0.0,0.000014,0.023,0.000014,0.000000,0.000398,0.000163,0.0,0.000222
2,Házi készítésű pappardelle tészta pesto rossoval,120perc,olcsó,közepes,"tészta, tészta alaprecept",110 perc,NaN,4,"[{'Ingredient': 'rétesliszt', 'Quantity': '165...",10.8,...,0.001,0.0,0.000022,0.075,0.000039,0.000000,0.000000,0.000002,0.0,0.000129
3,Hargita citromtortája,55perc,olcsó,könnyű,"édes süti, kevert sütemény",15 perc,40.0,8,"[{'Ingredient': 'vaj', 'Quantity': '125 g'}, {...",4.2,...,0.000,0.0,0.000014,0.045,0.000162,0.000000,0.000033,0.000002,0.0,0.000066
4,Tiramisu 1.,45perc,megfizethető,könnyű,"édes süti, tiramisu",45 perc,NaN,6,"[{'Ingredient': 'babapiskóta', 'Quantity': '40...",15.8,...,0.002,0.0,0.000028,0.139,0.000062,0.000006,0.000015,0.000006,0.0,0.000186


In [180]:
class IngredientScraper:
    """Class for scraping full details of an ingredient from Nosalty."""

    BASE_URL = "https://www.nosalty.hu/alapanyag/"

    def __init__(self, url):
        self.url = url
        self.soup = self.get_soup()

    @staticmethod
    def extract_urls_from_xml(file_path, check_url_format=False, date_range=None):
        """
        Extracts ingredient URLs from an XML file, optionally checking format and date range.

        Args:
            file_path (str): Path to the XML file.
            check_url_format (bool): If True, checks if the URL is valid.
            date_range (tuple): Optional (start_date, end_date) for filtering URLs by lastmod.
                                Format: ('YYYY-MM-DD', 'YYYY-MM-DD').

        Returns:
            list: List of valid URLs.
        """
        def is_valid_url(url):
            """Validates URL format."""
            pattern = r"^(http|https)://[^\s/$.?#].[^\s]*$"
            return re.match(pattern, url) is not None

        def is_within_date_range(date_text, start_date, end_date):
            """Checks if the date falls within the specified range."""
            try:
                date = datetime.strptime(date_text, "%Y-%m-%d")
                return start_date <= date <= end_date
            except ValueError:
                return False

        # Define XML namespace
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

        # Parse XML
        tree = ET.parse(file_path)
        root = tree.getroot()

        urls = []
        for url in root.findall('ns:url', namespace):
            loc = url.find('ns:loc', namespace)
            lastmod = url.find('ns:lastmod', namespace)

            if loc is not None:
                url_text = loc.text
                if not url_text.startswith(IngredientScraper.BASE_URL):
                    continue
                if check_url_format and not is_valid_url(url_text):
                    continue
                if date_range and lastmod is not None:
                    start_date, end_date = map(lambda d: datetime.strptime(d, "%Y-%m-%d"), date_range)
                    if not is_within_date_range(lastmod.text, start_date, end_date):
                        continue
                urls.append(url_text)

        return urls

    def get_soup(self):
        """Fetch and parse the ingredient webpage."""
        try:
            response = requests.get(self.url, timeout=5)
            if response.status_code != 200:
                return None
            return BeautifulSoup(response.text, 'html.parser')
        except requests.RequestException:
            return None

    def get_ingredient_name(self):
        """Extracts the ingredient name from metadata."""
        if not self.soup:
            return "N/A"
        meta_tag = self.soup.find("meta", property="og:title")
        return meta_tag["content"] if meta_tag and "content" in meta_tag.attrs else "Unknown Ingredient"

    def get_categories(self):
        """Extracts primary and secondary ingredient categories."""
        if not self.soup:
            return "N/A", "N/A"
        category_links = self.soup.find_all("a", class_="a-link -fontColorPrimary -fontSize-16 -recipe pr-3 pl-2")
        categories = [link.text.strip() for link in category_links]

        primary_category = categories[1] if len(categories) > 0 else "N/A"
        secondary_category = categories[2] if len(categories) > 2 else "N/A"
        return primary_category, secondary_category

    def get_nutrition_data(self):
        """Extracts nutrition details for the ingredient."""
        if not self.soup:
            return {}

        nutrition_data = {}

        # Locate the nutrition section
        ingredient_section = self.soup.find("div", id="ingredientCalculatorResult")
        if not ingredient_section:
            return {}  # No nutrition section found, return empty

        # Find all nutrition data rows
        rows = ingredient_section.find_all('div', class_='d-flex align-items-center flex-row w-100')

        for row in rows:
            # Extract labels and values
            paragraphs = row.find_all('p')
            if len(paragraphs) == 2:
                label = paragraphs[0].text.strip()
                value = paragraphs[1].text.strip()

                # Save to dictionary
                nutrition_data[label] = value

        return nutrition_data

    def to_dataframe(self):
        """Converts extracted ingredient details into a pandas DataFrame."""
        primary_category, secondary_category = self.get_categories()
        return pd.DataFrame([{
            "Ingredient Name": self.get_ingredient_name(),
            "Primary Category": primary_category,
            "Secondary Category": secondary_category,
            **self.get_nutrition_data()
        }])


In [191]:
# XML paths
file_paths = [
    r'C:\Users\Bálint\Desktop\Asztal\Projekt\inputs\receptek_url.xml',
    r'C:\Users\Bálint\Desktop\Asztal\Projekt\inputs\receptek_url_2.xml',
    r'C:\Users\Bálint\Desktop\Asztal\Projekt\inputs\receptek_url_3.xml'
]

# Processing all URLs 
all_ingredient_urls = []
for file_path in file_paths:
    urls_ingredients = IngredientScraper.extract_urls_from_xml(file_path, check_url_format=True)
    all_ingredient_urls.extend(urls_ingredients)

In [190]:
# Start measuring execution time
start_time = time.time()

# Initialize counters
ingredient_data = []
success_count = 0
total_count = len(all_ingredient_urls)

# Process each ingredient URL
with tqdm(total=total_count, desc="Processing Ingredients") as pbar:
    for i, url in enumerate(all_ingredient_urls, start=1):
        ingredient_scraper = IngredientScraper(url)
        ingredient_details_df = ingredient_scraper.to_dataframe()

        if not ingredient_details_df.empty:
            ingredient_data.append(ingredient_details_df)
            success_count += 1  # Count successful scrapes

        # Periodically print status updates
        if i % 150 == 0 or i == total_count:
            elapsed_time = time.time() - start_time
            print(f"Processed {i}/{total_count} URLs. Successful ingredients: {success_count}. Elapsed time: {elapsed_time:.2f} seconds.")

        pbar.update(1)  # Update progress bar

# Combine all ingredient data into a single DataFrame
final_ingredient_df = pd.concat(ingredient_data, ignore_index=True)

# Clean the extracted nutritional data
final_ingredient_df_cleaned = DataProcessor.clean_nutrition_data_ingredients(final_ingredient_df)

# Save the cleaned data to CSV
output_path = r"C:\Users\Bálint\Desktop\Asztal\Projekt\outputs\ingredients_cleaned.csv"
final_ingredient_df_cleaned.to_csv(output_path, index=False)

# End measuring execution time
end_time = time.time()
print(f"Total runtime: {end_time - start_time:.2f} seconds.")
# Display the first few rows of the cleaned dataset
final_ingredient_df_cleaned.head()

Processing Ingredients:  10%|█████▋                                                 | 151/1465 [01:44<09:38,  2.27it/s]

Processed 150/1465 URLs. Successful ingredients: 150. Elapsed time: 103.87 seconds.


Processing Ingredients:  20%|███████████▎                                           | 300/1465 [03:26<11:55,  1.63it/s]

Processed 300/1465 URLs. Successful ingredients: 300. Elapsed time: 206.73 seconds.


Processing Ingredients:  31%|████████████████▉                                      | 450/1465 [05:09<10:48,  1.56it/s]

...

Processed 1350/1465 URLs. Successful ingredients: 1350. Elapsed time: 900.80 seconds.


Processing Ingredients: 100%|██████████████████████████████████████████████████████| 1465/1465 [16:11<00:00,  1.51it/s]

Processed 1465/1465 URLs. Successful ingredients: 1465. Elapsed time: 971.72 seconds.


Total runtime: 973.58 seconds.


,Ingredient Name,Primary Category,Secondary Category,Foszfor,Magnézium,Nátrium,Kalcium,B6 vitamin,Kolin,Niacin - B3 vitamin,...,Béta-karotin,Lutein+zeaxantin,K vitamin,A vitamin,Likopin,Alfa-karotin,Béta-kriptoxantin,koleszterin,Retinol,Folsav
0,vargánya gomba,gombák,erdei gomba,0.086,0.009,0.005,0.003,0.104,0.0173,0.003607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,áfonya,gyümölcsök,bogyós gyümölcs,0.012,0.006,0.001,0.006,0.052,0.0060,0.000418,...,0.000032,0.000080,0.000019,0.000003,NaN,NaN,NaN,NaN,NaN,NaN
2,vöröskáposzta,zöldségek,káposztafélék,0.030,0.016,0.027,0.045,0.209,0.0171,0.000418,...,0.000670,0.000329,0.000038,0.000056,0.000020,NaN,NaN,NaN,NaN,NaN
3,citrom,gyümölcsök,déli gyümölcs,0.016,0.008,0.002,0.026,0.080,0.0051,0.000100,...,0.000003,0.000011,NaN,0.000001,NaN,0.000001,0.00002,NaN,NaN,NaN
4,sárgarépa,zöldségek,gyökérzöldség,0.035,0.012,0.069,0.033,0.138,0.0088,0.000983,...,0.008285,0.000256,0.000013,0.000835,0.000001,0.003477,NaN,NaN,NaN,NaN
